# Install and Import Dependencies

In [30]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
import sys
print(sys.executable)
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
from bs4 import BeautifulSoup
import requests
import re 

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121, 2.1.1, 2.1.1+cpu, 2.1.1+cu118, 2.1.1+cu121, 2.1.2, 2.1.2+cpu, 2.1.2+cu118, 2.1.2+cu121, 2.2.0, 2.2.0+cpu, 2.2.0+cu118, 2.2.0+cu121, 2.2.1, 2.2.1+cpu, 2.2.1+cu118, 2.2.1+cu121, 2.2.2, 2.2.2+cpu, 2.2.2+cu118, 2.2.2+cu121, 2.3.0, 2.3.0+cpu, 2.3.0+cu118, 2.3.0+cu121)
ERROR: No matching distribution found for torch==1.8.1+cu111


c:\Users\thath\anaconda3\envs\sa\python.exe


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [11]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\thath\anaconda3\envs\sa\python.exe


c:\Users\thath\anaconda3\envs\sa\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [13]:
result = model(tokens)

In [14]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [15]:
int(torch.argmax(result.logits))+1

4

# Collect Reviews

In [19]:
r = requests.get('https://uk.trustpilot.com/review/www.qatarairways.com')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_body.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [20]:
reviews

['1.6',
 'In the Flights search site category',
 'Visit this website',
 '3,491 total',
 '5-star',
 '13%',
 '4-star',
 '3%',
 '3-star',
 '2%',
 '2-star',
 '6%',
 '1-star',
 '76%',
 'I recently went online to book a flight with Qatar Airways. Logging onto the airline site I was given a price for a flight from Sharjah to London Heathrow. I then signed into my frequent flyer account and saw a promotion offering a discount of up to 12% if you paid by visa card. I started a new search using the visa link and found that the price quoted had been increased  for the flight from Sharjah and then discounted to the price I had already been quoted for payment using any payment method. When I contacted Qatar about the misleading advertising, the customer service rep ignored the content of my complaint and failed to address the issue of false advertising.',
 'Date of experience: 29 April 2024',
 'I booked a flight with Qatar Airways that arrives in Doha at 8:40 PM and departs as soon as possible on t

# Load Reviews into DataFrame and Score

In [21]:
import numpy as np
import pandas as pd

In [22]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [23]:
df['review'].iloc[0]

'1.6'

In [24]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [25]:
sentiment_score(df['review'].iloc[1])

4

In [26]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [27]:
df

,review,sentiment
0,1.6,5
1,In the Flights search site category,4
2,Visit this website,5
3,"3,491 total",1
4,5-star,5
...,...,...
67,Companies can ask for reviews via automatic in...,5
68,We use dedicated people and clever technology ...,5
69,Here are 8 tips for writing great reviews.,5
70,Verification can help ensure real people write...,4


In [28]:
df['review'].iloc[3]

'3,491 total'